In [ ]:
import pandas as pd
import plotly.express as px
import json
import plotly.graph_objects as go

In [ ]:
fragment_file = r'data/instances_regular/fragments/f-I-5-5-200-10.json'
fragment_file = r'data/instances_large/fragments/f-I-7-7-1000-01.json'
fragment_data = json.load(open(fragment_file))
# instance_data = json.load(open('data/instances_regular/I-1-1-50-01.json'))
print(len(fragment_data['fragments']), len(fragment_data['contracted_fragments']),  len(fragment_data['contracted_fragments'])/len(fragment_data['fragments']) *100)

In [ ]:
from visualiser import visualise_timed_network
from constants import TimedDepot

### Arrival cases


In [ ]:
# Create 3 timed depots with same id, 5 time units apart
timed_depots = [TimedDepot(time=0, id=0), TimedDepot(id=0, time=5), TimedDepot(time=10, id=0), TimedDepot(time=15, id=0)]
fig = visualise_timed_network(timed_depots, [], [{"timed_depot": a} for a in zip(timed_depots[:-1], timed_depots[1:])])

#compressed
fig = visualise_timed_network(timed_depots[:1]+timed_depots[-1:], [], [{"timed_depot": a} for a in zip(timed_depots[:-1], timed_depots[1:])])
inbound = False
for td in timed_depots:
    # inbound
    # if inbound:
    if td.time > 9:
        x, y, ax, ay = td.time+3, td.id+2, td.time, td.id
    else:
        x, y, ax, ay = td.time, td.id, td.time-3, td.id+2
    

    fig.add_annotation(
        x=x,
        y=y,
        ax=ax,
        ay=ay,
        xref='x',
        yref='y',
        axref='x',
        ayref='y',
        showarrow=True,
        arrowhead=2,
        arrowsize=2,  # Increase the size of the arrowhead
        arrowwidth=2,  # Increase the width of the arrow
        arrowcolor='#636363'
    )
    if td.time == 10:
        x, y, ax, ay = td.time, td.id, td.time-3, td.id+2
        # x, y, ax, ay = td.time+3, td.id+2, td.time, td.id
        fig.add_annotation(
            x=x,
            y=y,
            ax=ax,
            ay=ay,
            xref='x',
            yref='y',
            axref='x',
            ayref='y',
            showarrow=True,
            arrowhead=2,
            arrowsize=2,  # Increase the size of the arrowhead
            arrowwidth=2,  # Increase the width of the arrow
            arrowcolor='#636363'
        )

fig.write_image("images/time_space_compressed_two_in_one_correct.png")
fig.show()

In [4]:
from fragment_generation import *